In [129]:
import pandas as pd
import numpy as np
import re
import os
import shutup
shutup.please()

DATA PROCESS

In [141]:
# Path file data
excel_file = "D:\ml_chatbot_finance\data_0505.xlsx"

# Name of file md
nlu_file = "nlu_chatbot_finance_0505.md"


df_data = pd.read_excel(excel_file)
# with open(excel_file, 'r', encoding='utf16') as f:
#     df_data = pd.read_excel(f)
id_series = df_data.dropna(subset = ['ID'])['ID']
# print(df_data.dropna(subset = ['ID'])['ID'])
# for i in id_series:
#     print('1', i)

if os.path.isfile(nlu_file):
    os.remove(nlu_file)

## create nlu .md file
intent_list = df_data['ID'].unique()
for intent in intent_list:
    print('nlu', intent)
    df_id = df_data[df_data['ID'] == float(intent)]
    df_questions_dropna = df_id.dropna(subset=['questions'])['questions']
    with open(nlu_file, 'a') as f:
        f.write('## intent: q' + str(intent) + '\n')
    for question in df_questions_dropna:
        question = question.lower()
        print(question, type(question))
        # question = sentense_clean(question)
        with open(nlu_file, 'a', encoding = 'utf-8') as f:
            # f.write('## intent:' + intent + '\n')
            f.write('- ' + question + '\n')
print(df_data['questions'])


nlu 1.0
hello <class 'str'>
xin chào <class 'str'>
chào <class 'str'>
hi <class 'str'>
nlu 2.0
bạn có khỏe không <class 'str'>
hôm nay bạn khỏe không <class 'str'>
how are you? <class 'str'>
nlu 3.0
good morning <class 'str'>
good afternoon <class 'str'>
good evening <class 'str'>
nlu 4.0
tôi muốn hỏi bạn <class 'str'>
tôi có vài câu hỏi <class 'str'>
tôi hỏi bạn được không <class 'str'>
tôi cần hỏi vài thứ <class 'str'>
tư vấn cho tôi <class 'str'>
nlu 5.0
cho tôi thông tin <class 'str'>
bạn biết thông tin gì <class 'str'>
nlu 6.0
bạn sẵn sàng chưa <class 'str'>
are you ready <class 'str'>
nlu 7.0
cảm ơn bạn <class 'str'>
cảm ơn <class 'str'>
thank you <class 'str'>
thanks <class 'str'>
nlu 8.0
hẹn gặp bạn sau <class 'str'>
see you <class 'str'>
bai <class 'str'>
bye <class 'str'>
bai bai <class 'str'>
pp <class 'str'>
tạm biệt <class 'str'>
nlu 9.0
bạn tên gì <class 'str'>
tên mày là gì <class 'str'>
mày tên gì <class 'str'>
what is your name <class 'str'>
nlu 101.0
tôi nên đầu tư ng

TRAIN NLU

In [142]:
import logging
import pprint
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer
from rasa_nlu.model import Interpreter
#from rasa_nlu.evaluate import run_evaluation

logfile = 'nlu_model.log'

def train_nlu(data_path, p_name, configs, model_path):
    logging.basicConfig(filename=logfile, level=logging.DEBUG)
    training_data = load_data(data_path)
    trainer = Trainer(config.load(configs))
    trainer.train(training_data)
    model_directory = trainer.persist(model_path, project_name=p_name, fixed_model_name='nlu')



In [145]:
path_data = nlu_file #name of file md
name_model = 'chatbot_ver2' #name of model
path_config = 'nlu_config.yml' #flie config
pathsave = r'D:\ml_chatbot_finance\model' #path_save_model
train_nlu(path_data, name_model, path_config, pathsave)

Epochs: 100%|██████████| 300/300 [00:12<00:00, 23.64it/s, loss=0.114, acc=1.000]


Inference

In [146]:
from rasa_nlu.model import Interpreter

In [82]:

def run_core(nlu_model_path, excel_file):
    logging.basicConfig(filename=logfile, level=logging.DEBUG)
    NLU_CONF = 0.5
    interpreter = Interpreter.load(nlu_model_path)


    df_data = pd.read_excel(excel_file, dtype={'ID': np.float})

    print("Your bot is ready to talk! Type your messages here or send 'stop'")
    while True:
        question = input('human:')
        if question == 'stop':
            break

        question = question.lower()
        id = interpreter.parse(question)

        intent = id["intent"]["name"]
        conf = id["intent"]["confidence"]
        if intent != None and conf >= NLU_CONF:
            df_id = df_data[df_data['ID'] == float(intent[1:])]
            df_res_dropna = df_id.dropna(subset=['response'])
            df_res_dropna['link'] = df_res_dropna['link'].fillna(0)
            full_res = ''
            for res in df_res_dropna.index:
                if df_res_dropna['link'][res] != 0:
                    full_res = full_res + df_res_dropna['response'][res] + '\n' + df_res_dropna['link'][res] + '\n'
                else:
                    full_res = full_res + df_res_dropna['response'][res] + '\n' 
        else:
            full_res = 'tôi chưa hiểu ý của bạn, bạn có thể nói rõ hơn không?'
        print("Bot:", full_res)
    #return agent



In [136]:
if __name__ == '__main__':
    logging.basicConfig(filename=logfile, level=logging.DEBUG)
    path_nlu_model = r"D:\ml_chatbot_finance\model\chatbot_ver1\nlu"
    path_data_full = r"D:\ml_chatbot_finance\data_0505.xlsx"
    run_core(path_nlu_model, path_data_full)
    print("end")

TypeError: run_core() missing 1 required positional argument: 'questionss'

Testing

In [147]:
import pandas as pd
from rasa_nlu.model import Interpreter
from sklearn.metrics import accuracy_score
import re


In [150]:

def test_nlu(nlu_path, excel_file, out_file):
    interpreter = Interpreter.load(nlu_path)
    df = pd.read_excel(excel_file)
    intent_list = df['ID'].unique()
    df_data = df[['questions','ID']].dropna()
    print(df_data.columns)
    pre_id0 = []
    pre_conf0 = []
    pre_id1 = []
    pre_conf1 = []
    pre_id2 = []
    pre_conf2 = []
    for i in df_data.index:
        ques = df_data['questions'][i]
        ques = ques.lower()
        # ques = sentense_clean(ques)
        print(ques)
        int_out = interpreter.parse(ques)
        print(int_out)
        if len(intent_list) == 3:
            for j in range(3):
                name = str(int(int_out['intent_ranking'][j]['name'][1:]))
                conf = float(int_out['intent_ranking'][j]['confidence'])
                if j == 0:
                    pre_id0.append(name)
                    pre_conf0.append(conf)
                elif j == 1:
                    pre_id1.append(name)
                    pre_conf1.append(conf)
                elif j == 2:
                    pre_id2.append(name)
                    pre_conf2.append(conf)
        elif len(intent_list) == 2:
            for j in range(2):
                name = str(int(int_out['intent_ranking'][j]['name'][1:]))
                conf = float(int_out['intent_ranking'][j]['confidence'])
                if j == 0:
                    pre_id0.append(name)
                    pre_conf0.append(conf)
                elif j == 1:
                    pre_id1.append(name)
                    pre_conf1.append(conf)
                pre_id2.append('None')
                pre_conf2.append(0.0)
        else:
            name = str(int(int_out['intent_ranking'][0]['name'][1:]))
            conf = float(int_out['intent_ranking'][0]['confidence'])        
            pre_id0.append(name)
            pre_conf0.append(conf)
            pre_id1.append('None')
            pre_conf1.append(0.0)            
            pre_id2.append('None')
            pre_conf2.append(0.0)
    df_data['ID0'] = pre_id0
    df_data['conf0'] = pre_conf0
    df_data['ID1'] = pd.Series(pre_id1)
    df_data['conf1'] = pd.Series(pre_conf1)
    df_data['ID2'] = pd.Series(pre_id2)
    df_data['conf2'] = pd.Series(pre_conf2)

    y_true = df_data['ID'].values.tolist()
    y_true = list(map(str,y_true))
    print(y_true, pre_id0)
    acc = accuracy_score(y_true, pre_id0)
    print(df_data, acc)
    df_data['accuracy'] = acc
    df_data.to_excel(out_file)


In [153]:
if __name__ == '__main__':

    nlu_path = r"D:\ml_chatbot_finance\model\chatbot_ver2\nlu"
    excel_file = r"D:\ml_chatbot_finance\data_0505.xlsx"
    out_file = r"D:\ml_chatbot_finance\evaluation\eval_data_0505.xlsx"
    test_nlu(nlu_path, excel_file, out_file)
    print("end")

Index(['questions', 'ID'], dtype='object')
hello
{'intent': {'name': 'q1.0', 'confidence': 0.9431260228157043}, 'entities': [], 'intent_ranking': [{'name': 'q1.0', 'confidence': 0.9431260228157043}, {'name': 'q53.0', 'confidence': 0.246758371591568}, {'name': 'q255.0', 'confidence': 0.22328147292137146}, {'name': 'q201.0', 'confidence': 0.2145199328660965}, {'name': 'q260.0', 'confidence': 0.19883745908737183}, {'name': 'q202.0', 'confidence': 0.1935087889432907}, {'name': 'q106.0', 'confidence': 0.17204831540584564}, {'name': 'q259.0', 'confidence': 0.1222296729683876}, {'name': 'q59.0', 'confidence': 0.11421170830726624}, {'name': 'q111.0', 'confidence': 0.10900861024856567}], 'text': 'hello'}
xin chào
{'intent': {'name': 'q1.0', 'confidence': 0.9639406800270081}, 'entities': [], 'intent_ranking': [{'name': 'q1.0', 'confidence': 0.9639406800270081}, {'name': 'q255.0', 'confidence': 0.19533684849739075}, {'name': 'q260.0', 'confidence': 0.18494528532028198}, {'name': 'q106.0', 'confid

In [98]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [111]:
Y = vectorizer.transform(['Nên chọn đầu tư chứng khoán dài hạn hay ngắn hạn?'])

<1x116 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']

In [99]:
print(vectorizer.get_feature_names_out())

['ai' 'bao' 'biệt' 'bắt' 'cao' 'cho' 'chính' 'chóng' 'chọn' 'chứng' 'có'
 'công' 'cấp' 'cần' 'cổ' 'của' 'doanh' 'dài' 'dịch' 'giao' 'giá' 'giữa'
 'gì' 'hay' 'hiểu' 'huy' 'hơn' 'hưởng' 'hạn' 'hợp' 'khi' 'khoán' 'khác'
 'khái' 'không' 'kiếm' 'loại' 'là' 'làm' 'lợi' 'một' 'nghiệp' 'nghĩa'
 'ngắn' 'nhanh' 'nhau' 'nhiêu' 'nhuận' 'như' 'những' 'niệm' 'nào' 'nên'
 'phiếu' 'phân' 'phù' 'phải' 'quan' 'quyết' 'ro' 'rủi' 'sao' 'sách' 'sản'
 'số' 'sổ' 'sự' 'thì' 'thấp' 'thế' 'thể' 'thị' 'thực' 'tiền' 'trong'
 'trái' 'trò' 'trước' 'trường' 'trị' 'trọng' 'trực' 'tuyến' 'tuổi' 'ty'
 'tài' 'tìm' 'tính' 'tôi' 'tư' 'tại' 'tế' 'tố' 'vai' 'việc' 'và' 'vào'
 'vốn' 'với' 'xem' 'xác' 'xét' 'yếu' 'ích' 'điều' 'điểm' 'đâu' 'được'
 'đầu' 'đến' 'để' 'định' 'đối' 'độ' 'động' 'ảnh']


In [101]:
print(X.toarray()[1])

[0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 2 1 0 1 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0]


In [ ]:
cosine_similarity()

In [104]:

# import required libraries
import numpy as np
from numpy.linalg import norm

In [127]:
A = X.toarray()[5]
B = Y.toarray()[0]

In [128]:
cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

Cosine Similarity: 0.2773500981126146


In [123]:
1.00 + 0.85 + 0.81 + 0.73 + 0.849

4.239

In [124]:
4.239/5

0.8478